# Dataset
https://www.kaggle.com/puneetbhaya/online-retail/

## Spark Init

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("simple fp-growth").getOrCreate()

In [4]:
print(spark)

## Load Dataset

In [5]:
import pandas

In [6]:
data = pandas.read_excel("D:\Repos\Resource\Online-Retail\online-retail.xlsx")

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [9]:
#convert data to str to prevent error
data[['InvoiceNo','StockCode','Description']]=data[['InvoiceNo','StockCode','Description']].astype(str)

In [11]:
#create dataframe
df = spark.createDataFrame(data)

In [124]:
#create dataframe again ,since it already changed at In 30
backup_df = spark.createDataFrame(data)

In [12]:
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [30]:
#select just two column of it
df = df.selectExpr(['InvoiceNo as id','StockCode as items'])

In [31]:
df.show()

+------+------+
|    id| items|
+------+------+
|536365|85123A|
|536365| 71053|
|536365|84406B|
|536365|84029G|
|536365|84029E|
|536365| 22752|
|536365| 21730|
|536366| 22633|
|536366| 22632|
|536367| 84879|
|536367| 22745|
|536367| 22748|
|536367| 22749|
|536367| 22310|
|536367| 84969|
|536367| 22623|
|536367| 22622|
|536367| 21754|
|536367| 21755|
|536367| 21777|
+------+------+
only showing top 20 rows



In [34]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- items: string (nullable = true)



## Mapping Dataset

In [18]:
from pyspark.sql.functions import collect_list

In [35]:
#aggregate it into 1 column by its id
df_mapped = df.groupby("id").agg(collect_list('items').alias('items'))

In [36]:
df_mapped.show()

+-------+--------------------+
|     id|               items|
+-------+--------------------+
| 536596|[21624, 22900, 22...|
| 536938|[22386, 85099C, 2...|
| 537252|             [22197]|
| 537691|[22791, 22171, 82...|
| 538041|             [22145]|
| 538184|[22585, 21481, 22...|
| 538517|[22491, 21232, 21...|
| 538879|[84819, 22150, 21...|
| 539275|[22909, 22423, 22...|
| 539630|[21484, 85099B, 2...|
| 540499|[21868, 22697, 22...|
| 540540|[21877, 21868, 21...|
| 540976|[22394, 21890, 22...|
| 541432|[21485, 22457, 84...|
| 541518|[21880, 21881, 21...|
| 541783|[22423, 22854, 22...|
|C540850|             [21231]|
| 542026|[21754, 82600, 22...|
| 542375|[21731, 22367, 22...|
| 543641|[85123A, 21833, 2...|
+-------+--------------------+
only showing top 20 rows



## FP Growth

In [24]:
from pyspark.ml.fpm import FPGrowth

In [40]:
# try to train it, even though it is error since there is duplicate value on a row in Items, solution is at In 49
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(df_mapped)

Py4JJavaError: An error occurred while calling o341.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 30.0 failed 1 times, most recent failure: Lost task 3.0 in stage 30.0 (TID 904, localhost, executor driver): org.apache.spark.SparkException: Items in a transaction must be unique but got WrappedArray(22974, 22972, 21929, 85099F, 22385, 22899, 21169, 22382, 22634, 22804, 22635, 20981, 22637, 21871, 82482, 22637, 22726, 22727, 22551, 22557, 85232B, 22630, 22629).
	at org.apache.spark.mllib.fpm.FPGrowth$$anonfun$5.apply(FPGrowth.scala:243)
	at org.apache.spark.mllib.fpm.FPGrowth$$anonfun$5.apply(FPGrowth.scala:240)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:434)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:191)
	at org.apache.spark.shuffle.sort.SortShuff

In [45]:
from pyspark.sql.types import ArrayType, StringType

In [49]:
#remove duplicates since a cashier and its app might see a product that inserted twice as two different rows

from pyspark.sql.functions import udf

remove_dupes_from_array = udf(lambda row: list(set(row)), ArrayType(StringType()))
df_mapped = df_mapped.withColumn("items_without_dupes", remove_dupes_from_array("items"))

In [56]:
df_mapped.show()

+-------+--------------------+----------------------+--------------------+
|     id|               items|arraycol_without_dupes| items_without_dupes|
+-------+--------------------+----------------------+--------------------+
| 536596|[21624, 22900, 22...|  [84926A, 21624, 2...|[84926A, 21624, 2...|
| 536938|[22386, 85099C, 2...|  [21479, 84997B, 2...|[21479, 84997B, 2...|
| 537252|             [22197]|               [22197]|             [22197]|
| 537691|[22791, 22171, 82...|  [22505, 22791, 82...|[22505, 22791, 82...|
| 538041|             [22145]|               [22145]|             [22145]|
| 538184|[22585, 21481, 22...|  [22492, 22561, 48...|[22492, 22561, 48...|
| 538517|[22491, 21232, 21...|  [22197, 22844, 22...|[22197, 22844, 22...|
| 538879|[84819, 22150, 21...|  [22130, 22555, 84...|[22130, 22555, 84...|
| 539275|[22909, 22423, 22...|  [22423, 21914, 22...|[22423, 21914, 22...|
| 539630|[21484, 85099B, 2...|  [22988, 84347, 22...|[22988, 84347, 22...|
| 540499|[21868, 22697, 2

In [57]:
#select just two of the column
df_mapped=df_mapped.selectExpr(['id','items_without_dupes as items'])

In [58]:
df_mapped.show()

+-------+--------------------+
|     id|               items|
+-------+--------------------+
| 536596|[84926A, 21624, 2...|
| 536938|[21479, 84997B, 2...|
| 537252|             [22197]|
| 537691|[22505, 22791, 82...|
| 538041|             [22145]|
| 538184|[22492, 22561, 48...|
| 538517|[22197, 22844, 22...|
| 538879|[22130, 22555, 84...|
| 539275|[22423, 21914, 22...|
| 539630|[22988, 84347, 22...|
| 540499|[21755, 84978, 22...|
| 540540|[22555, 22551, 22...|
| 540976|[22207, 21110, 84...|
| 541432|[22113, 22457, 21...|
| 541518|[20724, 21982, 20...|
| 541783|[22197, 84978, 22...|
|C540850|             [21231]|
| 542026|[22197, 22398, 22...|
| 542375|[22367, 22629, 21...|
| 543641|[22371, 44265, 21...|
+-------+--------------------+
only showing top 20 rows



In [65]:
df_mapped.printSchema()

root
 |-- id: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [59]:
# retrain
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(df_mapped)

In [60]:
# Display frequent itemsets.
model.freqItemsets.show()

+-----+----+
|items|freq|
+-----+----+
+-----+----+



In [63]:
# since it show no result, let's lower the minimum

fpGrowth2 = FPGrowth(itemsCol="items", minSupport=0.3, minConfidence=0.3)
model2 = fpGrowth2.fit(df_mapped)

In [64]:
# Display frequent itemsets.
model2.freqItemsets.show()

+-----+----+
|items|freq|
+-----+----+
+-----+----+



In [74]:
# still no result, let's lower it more
fpGrowth3 = FPGrowth(itemsCol="items", minSupport=0.05, minConfidence=0.05)
model3 = fpGrowth3.fit(df_mapped)

In [75]:
# Display frequent itemsets.
model3.freqItemsets.show()

+--------+----+
|   items|freq|
+--------+----+
|[85123A]|2246|
| [22423]|2172|
|[85099B]|2135|
| [47566]|1706|
| [20725]|1608|
| [84879]|1468|
| [22720]|1462|
| [22197]|1442|
| [21212]|1334|
| [22383]|1306|
| [20727]|1295|
+--------+----+



In [76]:
# Display generated association rules.
model3.associationRules.show()

+----------+----------+----------+
|antecedent|consequent|confidence|
+----------+----------+----------+
+----------+----------+----------+



In [79]:
# got frequent but no rules, let's lower it more
fpGrowth4 = FPGrowth(itemsCol="items", minSupport=0.01, minConfidence=0.01)
model4 = fpGrowth4.fit(df_mapped)

In [80]:
# Display frequent itemsets.
model4.freqItemsets.show()

+----------------+----+
|           items|freq|
+----------------+----+
|         [22633]| 487|
|         [23236]| 344|
|        [85123A]|2246|
|         [22423]|2172|
| [22423, 85123A]| 355|
|         [22667]| 486|
|         [22579]| 343|
|  [22579, 22578]| 282|
|        [85099B]|2135|
| [85099B, 22423]| 288|
|[85099B, 85123A]| 404|
|         [22620]| 486|
|        [84536A]| 342|
|         [71053]| 342|
|         [47566]|1706|
| [47566, 85099B]| 332|
|  [47566, 22423]| 398|
| [47566, 85123A]| 391|
|         [85150]| 483|
|         [20725]|1608|
+----------------+----+
only showing top 20 rows



In [82]:
# Display generated association rules.
model4.associationRules.show()

+--------------+----------+-------------------+
|    antecedent|consequent|         confidence|
+--------------+----------+-------------------+
|       [22554]|   [22551]| 0.4823695345557123|
|       [22554]|   [22556]| 0.3991537376586742|
|       [22960]|   [21212]|0.21885245901639344|
|       [22960]|  [85099B]|0.23688524590163934|
|       [22960]|   [22423]|0.23852459016393443|
|       [22960]|   [22720]| 0.3155737704918033|
|       [22960]|   [22961]|0.38934426229508196|
|       [22960]|   [22666]|0.28032786885245903|
|       [22960]|   [22993]| 0.2540983606557377|
|       [22960]|   [22697]|0.21475409836065573|
|       [22960]|   [22722]|0.22131147540983606|
|[20726, 22382]|   [20728]|  0.546583850931677|
|[20726, 22382]|   [20725]| 0.6356107660455487|
|[20726, 22382]|   [20727]| 0.5445134575569358|
|[20726, 22382]|   [22383]| 0.5403726708074534|
|       [21977]|   [21212]| 0.4948571428571429|
|       [21977]|   [84991]| 0.4045714285714286|
|       [22699]|   [22423]|0.47946428571

In [86]:
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model4.transform(df_mapped).show()

+-------+--------------------+--------------------+
|     id|               items|          prediction|
+-------+--------------------+--------------------+
| 536596|[84926A, 21624, 2...|[23355, 22112, 22...|
| 536938|[21479, 84997B, 2...|[85099B, 20725, 2...|
| 537252|             [22197]|[85099B, 20725, 2...|
| 537691|[22505, 22791, 82...|[21212, 85099B, 2...|
| 538041|             [22145]|                  []|
| 538184|[22492, 22561, 48...|[85099B, 85123A, ...|
| 538517|[22197, 22844, 22...|[85099B, 20725, 2...|
| 538879|[22130, 22555, 84...|[84991, 85099B, 2...|
| 539275|[22423, 21914, 22...|[85123A, 85099B, ...|
| 539630|[22988, 84347, 22...|[85123A, 47566, 2...|
| 540499|[21755, 84978, 22...|[23170, 22698, DO...|
| 540540|[22555, 22551, 22...|[22112, 21485, 22...|
| 540976|[22207, 21110, 84...|[22556, 21931, 20...|
| 541432|[22113, 22457, 21...|[85099B, 85123A, ...|
| 541518|[20724, 21982, 20...|[21931, 20725, 22...|
| 541783|[22197, 84978, 22...|[23170, 22698, DO...|
|C540850|   

In [96]:
df2=spark.createDataFrame([
    ('0',['21232'])
],['id','items'])

In [128]:
#get the description of the testing product
backup_df.select('Description').filter(backup_df.StockCode=='21232').show(1)

+--------------------+
|         Description|
+--------------------+
|STRAWBERRY CERAMI...|
+--------------------+
only showing top 1 row



In [97]:
# get the prediction
model4.transform(df2).show()

+---+-------+----------+
| id|  items|prediction|
+---+-------+----------+
|  0|[21232]|   [21231]|
+---+-------+----------+



In [118]:
# collect the prediction
cobadeh = model4.transform(df2).select('prediction').collect()[0][0]

In [120]:
print(cobadeh[0])

21231


In [126]:
# get the prediction's description
backup_df.select('Description').filter(backup_df.StockCode==cobadeh[0]).show(1)

+--------------------+
|         Description|
+--------------------+
|SWEETHEART CERAMI...|
+--------------------+
only showing top 1 row



## Conclusion

Based on this Datasets and the Test above, we can conclude that the one who bought "Strawberry Ceramic" are likely to buy "Sweetheart Ceramic" also.

## Reference
1. (Remove Duplicates) https://stackoverflow.com/questions/54185710/remove-duplicates-from-pyspark-array-column
2.